In [3]:
# Web3 Trading Analysis - Fear & Greed vs Trader Behavior

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load datasets
fear_greed_df = pd.read_csv("fear_greed_index.csv")
historical_df = pd.read_csv("historical_data.csv")

# Convert Timestamp to datetime and extract date
historical_df['Timestamp IST'] = pd.to_datetime(historical_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
historical_df['date'] = historical_df['Timestamp IST'].dt.date

# Aggregate trader metrics by date
agg_historical = historical_df.groupby('date').agg({
    'Execution Price': 'mean',
    'Size USD': ['sum', 'mean'],
    'Closed PnL': ['sum', 'mean'],
    'Start Position': 'mean',
    'Fee': 'mean',
    'Account': 'nunique'
}).reset_index()

# Flatten column names
agg_historical.columns = ['date', 'avg_execution_price', 'total_volume_usd', 'avg_volume_usd',
                          'total_pnl', 'avg_pnl', 'avg_position', 'avg_fee', 'unique_traders']

# Convert date columns to datetime
agg_historical['date'] = pd.to_datetime(agg_historical['date'])
fear_greed_df['date'] = pd.to_datetime(fear_greed_df['date'])

# Merge datasets
merged_df = pd.merge(agg_historical, fear_greed_df[['date', 'classification', 'value']], on='date', how='left')

# Create directories if they don't exist
os.makedirs('csv_files', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

# Save processed data
merged_df.to_csv("csv_files/processed_data.csv", index=False)

# Create output charts
def save_chart(fig, filename):
    fig.savefig(f"outputs/{filename}", bbox_inches='tight')
    plt.close(fig)

# 1. Volume vs Sentiment
fig, ax = plt.subplots(figsize=(10,6))
sns.boxplot(data=merged_df, x='classification', y='total_volume_usd', palette='coolwarm', ax=ax)
ax.set_title('Total Volume by Sentiment')
save_chart(fig, "volume_vs_sentiment.png")

# 2. Avg PnL vs Sentiment
fig, ax = plt.subplots(figsize=(10,6))
sns.boxplot(data=merged_df, x='classification', y='avg_pnl', palette='vlag', ax=ax)
ax.set_title('Average PnL by Sentiment')
save_chart(fig, "pnl_vs_sentiment.png")

# 3. Unique Traders vs Sentiment Index
fig, ax = plt.subplots(figsize=(10,6))
sns.scatterplot(data=merged_df, x='value', y='unique_traders', hue='classification', palette='Set2', ax=ax)
ax.set_title('Unique Traders vs Fear & Greed Index')
save_chart(fig, "traders_vs_index.png")

/tmp/ipython-input-3-980723485.py:51: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=merged_df, x='classification', y='total_volume_usd', palette='coolwarm', ax=ax)
/tmp/ipython-input-3-980723485.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=merged_df, x='classification', y='avg_pnl', palette='vlag', ax=ax)
